This notebook is part of [**Byron v0.1**](https://github.com/cad-polito-it/byron)  
Copyright 2023 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: [Apache-2.0](https://www.tldrlegal.com/license/apache-license-2-0-apache-2-0) 

In [1]:
import byron

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_12219/1429530752.py:1: ByronPerformanceWarning: 
  Paranoia checks are enabled in this notebook: performances can be significantly impaired
  [see https://github.com/cad-polito-it/byron/blob/pre-alpha/docs/paranoia.md for details]
  import byron


Everything for the definition of the constraints (the old "instruction library") is under the namespace `byron.framework` (lazy fingers can use `byron.f`)

# Frames

In [2]:
macro_foo = byron.f.macro('foo #{num:04x}', num=byron.f.integer_parameter(0, 2**16))
macro_bar = byron.f.macro('bar {num:+0.3e}', num=byron.f.float_parameter(-1, 1))
macro_baz = byron.f.macro('baz 0x{array}', array=byron.f.array_parameter("0123456789abcdef", 8))

## Macro bunches

Macro bunches are sequences of macros randomly selected from a bunch.

In [3]:
bunch = byron.f.bunch([macro_foo, macro_bar, macro_baz], size=(5, 10 + 1))
byron.f.as_text(bunch)

; 🖋 n1 ➜ Frame❬MacroBunch#1❭
baz 0x1b318fbc  ; 🖋 n1.n2 ➜ Macro❬User#3❭
bar +5.721e-01  ; 🖋 n1.n3 ➜ Macro❬User#2❭
bar -7.438e-01  ; 🖋 n1.n4 ➜ Macro❬User#2❭
bar -9.923e-02  ; 🖋 n1.n5 ➜ Macro❬User#2❭
baz 0x852eca6d  ; 🖋 n1.n6 ➜ Macro❬User#3❭


The relative frequencies of the macros may be tweaked

In [4]:
bunch = byron.f.bunch([macro_foo, macro_bar, macro_baz], size=20, weights=[10, 2, 1])
byron.f.as_text(bunch)

AssertionError: ValueError (paranoia check): invalid value: 32791

## Frames

In [ ]:
all_foo = byron.f.bunch([macro_foo], size=3)
all_bar = byron.f.bunch([macro_bar], size=3)
mix = byron.f.bunch([macro_foo, macro_bar, macro_baz], size=(2, 5 + 1))

### Sequences

Create all frames in the sequence

In [ ]:
seq = byron.f.sequence(['begin', all_foo, 'break', all_bar, 'end'])
byron.f.as_text(seq)

Given the top frame, it is possible to visualize the forest with the syntactic structure (*as_forest*). **Note**: The text is the result of a [depth-first search](https://en.wikipedia.org/wiki/Depth-first_search) of the syntactic structure. 

It is also possible to visualize a frame as a, perhaps more intuitive, linear-genetic-programming genome (*as_lgp*) 

In [ ]:
byron.f.as_forest(seq)
byron.f.as_lgp(seq)

### Alternatives

Randomly chose a frame from the different alternatives

In [ ]:
alt = byron.f.alternative([macro_foo, macro_bar, all_foo, all_bar, 'dummy'])

for n in range(5):
    byron.f.as_text(alt, seed=n)

### Nesting Sequences and Alternatives

In [ ]:
all_foo = byron.f.bunch([macro_foo], size=3)
all_bar = byron.f.bunch([macro_bar], size=3)
sequence1 = byron.f.sequence(['; seq1 ----------------------', all_foo, all_bar])
sequence2 = byron.f.sequence(['; seq2 ----------------------', all_bar, all_foo])
alternative = byron.f.alternative([sequence1, sequence2])
final_sequence = byron.f.sequence(
    ['; sseq1 =====================', alternative, '; sseq2 =====================', alternative, alternative]
)

In [ ]:
byron.f.as_text(final_sequence)
byron.f.as_forest(final_sequence)
byron.f.as_lgp(final_sequence)

## Frame Names and String Magic

It is possible to specify a (unique) user name for Frames. The name can then be used to specify the frame.

In [ ]:
all_foo = byron.f.bunch([macro_foo], size=3, name='Foo')
all_bar = byron.f.bunch([macro_bar], size=3, name='Bar')

In [ ]:
all_foo == all_bar, all_foo == 'Foo', all_foo == 'Bar'

In [ ]:
set([all_foo, 'Foo', 'Bar', all_bar])